# Qiskit Coding Exercise: BB84 Quantum Key Distribution

This notebook guides you through simulating the BB84 protocol to securely distribute a cryptographic key. We will simulate Alice, Bob, and an eavesdropper (Eve) to see how quantum mechanics guarantees security.

**Setup:**
You will need `qiskit` (v1.0+), `qiskit-aer` and `numpy`.
`pip install qiskit qiskit-aer numpy`

In [8]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator

# Set a seed for reproducibility
np.random.seed(42)

--- 

## Phase 1: Engage (The Setup)

### 1. Generating Randomness
Alice needs two random strings of bits:
1.  **Data Bits:** The actual key material (0s and 1s).
2.  **Basis Bits:** The choice of basis for each bit.
    * 0 = **Z-Basis** (Rectilinear)
    * 1 = **X-Basis** (Diagonal)

**Task:** Generate 100 random data bits and 100 random basis bits for Alice.

In [9]:
n_bits = 100

# --- Your Code Here ---
alice_data = np.random.randint(0, 2, n_bits)
alice_bases = np.random.randint(0, 2, n_bits)
# ----------------------

print(f"Alice's first 10 Data Bits: {alice_data[:10]}")
print(f"Alice's first 10 Basis Bits: {alice_bases[:10]}")

Alice's first 10 Data Bits: [0 1 0 0 0 1 0 0 0 1]
Alice's first 10 Basis Bits: [0 1 1 1 1 1 1 1 1 0]


--- 

## Phase 2: Explore (Transmission)

### 2. Encoding Qubits
Now Alice prepares qubits based on her choices. 
* **Z-Basis (0):** Data 0 $\to |0\rangle$, Data 1 $\to |1\rangle$
* **X-Basis (1):** Data 0 $\to |+\rangle$, Data 1 $\to |-\rangle$

**Task:** Create a list of `QuantumCircuit` objects representing the stream of photons Alice sends.

In [10]:
def encode_qubits(data_bits, basis_bits):
    circuits = []
    for i in range(len(data_bits)):
        qc = QuantumCircuit(1, 1)
        data = data_bits[i]
        basis = basis_bits[i]
        
        # --- Your Code Here ---
        if basis == 0: # Z-Basis
            if data == 1:
                qc.x(0) # |1>
        else: # X-Basis
            if data == 0:
                qc.h(0) # |+>
            else:
                qc.x(0)
                qc.h(0) # |->
        # ----------------------
        circuits.append(qc)
    return circuits

alice_circuits = encode_qubits(alice_data, alice_bases)

### 3. Bob's Measurement
Bob receives the qubits. He *doesn't* know Alice's basis, so he chooses his own random bases to measure.

**Task:** Generate `bob_bases`. Then, simulate Bob measuring Alice's circuits. 
* If Bob chooses Z (0), measure directly.
* If Bob chooses X (1), apply H then measure.

In [11]:
sim = AerSimulator()   # your backend

def measure_qubits(circuits, measurement_bases):
    results = []

    for i, qc in enumerate(circuits):
        basis = measurement_bases[i]

        # Copy circuit
        measured_qc = qc.copy()

        # Ensure 1 classical bit exists
        if measured_qc.num_clbits == 0:
            measured_qc.add_clbits(1)

        # Apply basis rotation if Bob measures in X
        if basis == 1:
            measured_qc.h(0)

        # Perform measurement
        measured_qc.measure(0, 0)

        # Transpile for Aer
        tqc = transpile(measured_qc, sim)

        # Run simulation (1 shot)
        result = sim.run(tqc, shots=1).result()
        counts = result.get_counts()

        # Extract the 0/1 bit result
        bit = int(list(counts.keys())[0])
        results.append(bit)

    return np.array(results)

bob_bases = np.random.randint(0, 2, n_bits)
bob_results = measure_qubits(alice_circuits, bob_bases)
print("Bob's first 10 Results:", bob_results[:10])

Bob's first 10 Results: [0 1 1 1 0 1 0 0 0 1]


--- 

## Phase 3: Explain (Sifting)

### 4. Sifting the Key
Alice and Bob communicate their *bases* (not the data). They discard bits where `alice_bases != bob_bases`.

**Task:** Implement the sifting logic to produce the `raw_key`.

In [12]:
def sift_key(alice_bases, bob_bases, bob_results):
    sifted_key = []
    # --- Your Code Here ---
    for i in range(len(alice_bases)):
        if alice_bases[i] == bob_bases[i]:
            sifted_key.append(bob_results[i])
    # ----------------------
    return np.array(sifted_key)

raw_key = sift_key(alice_bases, bob_bases, bob_results)
print(f"Raw Key Length: {len(raw_key)} (Expected ~50)")
print(f"Raw Key: {raw_key}")

Raw Key Length: 47 (Expected ~50)
Raw Key: [0 1 0 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 1
 0 0 1 1 1 1 1 1 1 1]


--- 

## Phase 4: Elaborate (Eve's Attack)

### 5. Simulating Interception
Now, let's introduce Eve. She intercepts the qubits *before* Bob gets them. She measures in a random basis and resends the result.

**Task:** 
1.  Generate `eve_bases`.
2.  Eve measures Alice's circuits to get `eve_results`.
3.  Eve encodes her results into *new* circuits (`eve_circuits`).
4.  Bob measures `eve_circuits` instead of Alice's.

In [13]:
# 1. Eve's random bases
eve_bases = np.random.randint(0, 2, n_bits)

# 2. Eve intercepts & measures
eve_results = measure_qubits(alice_circuits, eve_bases)

# 3. Eve Resends (Encodes her results)
eve_resent_circuits = encode_qubits(eve_results, eve_bases)

# 4. Bob measures Eve's circuits
bob_results_attacked = measure_qubits(eve_resent_circuits, bob_bases)

# 5. Sift again (Alice & Bob don't know Eve was there yet!)
raw_key_attacked = sift_key(alice_bases, bob_bases, bob_results_attacked)
alice_key_part = sift_key(alice_bases, bob_bases, alice_data) # Alice's version of the sifted key

### 6. Calculate QBER (Error Check)
Alice and Bob compare a sample of their keys to calculate the Quantum Bit Error Rate.

**Task:** Calculate the percentage of bits that disagree between `raw_key_attacked` and `alice_key_part`.

In [14]:
errors = 0
for i in range(len(raw_key_attacked)):
    if raw_key_attacked[i] != alice_key_part[i]:
        errors += 1

qber = errors / len(raw_key_attacked)
print(f"QBER with Eve: {qber:.2%} (Theoretical: ~25%)")

QBER with Eve: 21.28% (Theoretical: ~25%)


--- 

## Phase 5: Evaluate (Post-Processing)

### 7. Error Reconciliation (Parity Check)
As explained in your notes, Alice and Bob split their key into blocks and compare parities to find errors. Let's simulate a simple 1-block parity check.

**Task:** Calculate the parity (sum modulo 2) of Alice's raw key and Bob's raw key. Do they match?

In [15]:
def calculate_parity(bit_string):
    return sum(bit_string) % 2

alice_parity = calculate_parity(alice_key_part)
bob_parity = calculate_parity(raw_key_attacked)

print(f"Alice's Parity: {alice_parity}")
print(f"Bob's Parity:   {bob_parity}")

if alice_parity != bob_parity:
    print("Mismatch detected! Errors present.")
else:
    print("Parities match (Warning: Even number of errors could hide here).")

Alice's Parity: 1
Bob's Parity:   1
Parities match (Warning: Even number of errors could hide here).


### 8. Privacy Amplification
To wipe out Eve's partial knowledge, we shrink the key using a hash function. A simple hash is to XOR random subsets of the key.

**Task:** Create a new 1-bit final key by XORing all the bits in the raw key together.

In [16]:
# Simple Privacy Amplification: XOR all bits to get 1 final secure bit
final_secure_bit_alice = 0
for bit in alice_key_part:
    final_secure_bit_alice ^= bit
    
final_secure_bit_bob = 0
for bit in raw_key_attacked:
    final_secure_bit_bob ^= bit

print(f"Alice's Final Key: {final_secure_bit_alice}")
print(f"Bob's Final Key:   {final_secure_bit_bob}")

Alice's Final Key: 1
Bob's Final Key:   1
